# **Example PSI Model Synthesis Notebook**

Tom Schad 

----

- This notebook does an example synthesis of the Global corona using a Predictive Sciences MHD model. 


----

In [1]:
import pycelp 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import os
os.environ["XUVTOP"] = '/usr/local/ssw/packages/chianti/dbase/'
import psi
import sunpy.coordinates.sun as sun
import sunpy.sun.constants as constants
plt.rcParams["image.origin"]='lower'   ## have to worry about flips? 
plt.rcParams["image.interpolation"]='nearest'
from scipy.interpolate import RegularGridInterpolator as rgi
from scipy.interpolate import interp1d
import multiprocessing
import tqdm

def euler_ry(alpha):
    '''Euler rotation matrix about y axis '''
    ry = np.array([[ np.cos(alpha), 0., np.sin(alpha)],
                   [            0., 1., 0.],
                   [-np.sin(alpha), 0., np.cos(alpha)]])
    return ry

def euler_rz(alpha):
    '''Euler rotation matrix about z axis '''
    rz = np.array([[ np.cos(alpha), -np.sin(alpha), 0.],
                   [ np.sin(alpha),  np.cos(alpha), 0.],
                   [            0.,             0., 1.]])
    return rz

## **Create an instance of the psi.Model class to load and interact with PSI coronal model data**

- Models used here were downloaded using this website from [Predictive Sciences](https://www.predsci.com/hmi/data_access.php)
- We picked the date of the US total solar eclipse (21 Aug 2017) using the med-cor-thermo2 model for this demonstration. 
- I had selected 12 UTC for the download time.  I'm not sure at this time where timestep information in the PSI models made be located.

In [2]:
## https://www.predsci.com/data/runs/cr2194-medium/hmi_mast_mas_std_0201/
## carrington rotation 2194
modelName = 'hmi__med-cor-thermo2-std01__med-hel-poly-std01'
corona = psi.Model('/home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/')

In [3]:
## return the instance name for some basic information on the class
corona

psi Model class
    ---------------------
    Data Directory Names: /home/tschad/Dropbox/psiData_21Aug2017_thermo2_12UTC/corona/
    Number of longitude samples: 181
    Number of latitude samples: 100
    Number of radial samples: 150
    Data shape: (181, 100, 150)
    
    Variables: 
    lons -- Longitudes [rad]
    lats -- Latitudes [rad]
    rs   -- Radial samples [solar radii units]
    temp -- temperature [K]
    ne -- electron density [cm^-3]
    br,bt,bp  -- Spherical components of magnetic field [G]
    bx,by,bz  -- Cartesian components of magnetic field [G]
    vr,vt,vp  -- Spherical components of velocity field [km/s]
    vx,vy,vz  -- Cartesian components of velocity field [km/s]
    bmag      -- total magnetic field intensity [B]
    thetaBlocal -- location inclination of magnetic field in solar frame [rad]
    

In [4]:
kk = 5

fig,ax = plt.subplots(2,2,figsize = (8,5))
ax = ax.flatten()
ax[0].imshow(corona.bmag[:,:,kk].T*np.cos(corona.thetaBlocal[:,:,kk]).T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[1].imshow(corona.thetaBlocal[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[2].imshow(corona.temp[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
ax[3].imshow(corona.ne[:,:,kk].T,extent = (corona.lons.min(),corona.lons.max(),corona.lats.min(),corona.lats.max()))
labels = 'Magnetogram','Local B Inclination','Temperature','Electron Density'
for n in range(4): ax[n].set_title(labels[n])

cbars = []
for axi in ax:
 cax = axi.inset_axes([1.04, 0.05, 0.05, 0.95], transform=axi.transAxes)
 cbar1 = fig.colorbar(axi.get_images()[0],ax=axi,cax=cax)
 cbars.append(cbar1)

fig.suptitle(modelName + '\nRadial Coordinate: ' + str(corona.rs[kk]))

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is the GONG 2194_000 synoptic map 

<img src="https://gong.nso.edu/data/magmap/QR/mqj/201708/mrmqj170830/mrmqj170830t0205c2194_000.jpg" />

## **Initialize the pyCELP model of the line or lines to be synthesized**

- Here we start with Fe XIII 1074 nm and keep the model fairly "small" at 50 levels.  The errors incurred by using reduced numbers of atomic levels is addressed in [Schad & Dima (2020)](https://rdcu.be/b5J2X) 


In [5]:
fe13 = pycelp.Ion('fe_13',nlevels = 50)

 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.elvlc
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.wgfa
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.scups
 reading:  /usr/local/ssw/packages/chianti/dbase/fe/fe_13/fe_13.psplups
 using default abundances: /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 reading:  /usr/local/ssw/packages/chianti/dbase/abundance/sun_photospheric_2009_asplund.abund
 testing default file: /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 reading:  /usr/local/ssw/packages/chianti/dbase/ioneq/chianti.ioneq
 setting up electron collision rate factors
 setting up proton  collision rate factors
 setting up non-dipole radiative rate factors
 getting non-dipole rate factors
 setting up dipole radiative rate factors


## **Calculate the statistical equilibrium for every cell**

- Here the statistical equilibrium is calculated for the Ion for each cell in the PSI model and the results of the upper level populations, alignments, and total ion population are stored in arrays. 
- The advantage of doing it this way is that that statistical equilibrium does not change with viewing angle, and so if you want to synthesis the polarized intensities at a different viewing angle, you do not have to recalculate the statistical equilibrium. 

In [6]:
def work(argin): 
    
    ii,jj,kk = argin 
    
    edens = corona.ne[ii,jj,kk]
    etemp = corona.temp[ii,jj,kk]
    ht = (corona.rs[kk]-1).clip(1.e-8)
    thetab = np.rad2deg(corona.thetaBlocal[ii,jj,kk])
    
    fe13.calc_rho_sym(edens,etemp,ht, thetab)
    
    ln1 = fe13.get_emissionLine(10747.)
    ln2 = fe13.get_emissionLine(10798.)
    
    return (ii,jj,kk,ln1.C_coeff,ln1.upper_level_alignment, 
                ln2.C_coeff,ln2.upper_level_alignment)
            
ncpus = multiprocessing.cpu_count()
arg_array = []
for ii in range(0,181,1): 
    for jj in range(0,100,1): 
        for kk in range(0,150,1): 
            arg_array.append([ii,jj,kk])
         
ccoef_10747 = np.zeros_like(corona.br)
align_10747 = np.zeros_like(corona.br)
ccoef_10798 = np.zeros_like(corona.br)
align_10798 = np.zeros_like(corona.br)

"""
p     = multiprocessing.Pool(processes=ncpus) 
rs    = p.imap(work,arg_array)
p.close()

for res in tqdm.tqdm(rs,total = len(arg_array)): 
    ii,jj,kk,tpop,r1,a1,r2,a2 = res 
    tot_ion_pop[ii,jj,kk] = tpop
    rho00_10747[ii,jj,kk] = r1
    align_10747[ii,jj,kk] = a1
    rho00_10798[ii,jj,kk] = r2
    align_10798[ii,jj,kk] = a2
    

"""

'\np     = multiprocessing.Pool(processes=ncpus) \nrs    = p.imap(work,arg_array)\np.close()\n\nfor res in tqdm.tqdm(rs,total = len(arg_array)): \n    ii,jj,kk,tpop,r1,a1,r2,a2 = res \n    tot_ion_pop[ii,jj,kk] = tpop\n    rho00_10747[ii,jj,kk] = r1\n    align_10747[ii,jj,kk] = a1\n    rho00_10798[ii,jj,kk] = r2\n    align_10798[ii,jj,kk] = a2\n    \n\n'

## **Setup forward synthesis geometry**

- This is set up to synthesize a FOV from an arbitrary observer's position, def

### Observer's Position

In [7]:
## not yet sure what to use here?  How to get fractional carrington rotation of the PSI data?
crn        = 2194.1  
crt        = sun.carrington_rotation_time(crn)
obsLon     = sun.L0(crt).rad
obsLat     = sun.B0(crt).rad
Obs_Sun_AU = sun.earth_distance(crt).value
rsunarc    = sun.angular_radius(crt).value  ## radius of sun in arcseconds .. later replace with sun ephemeris

print('Carrington Rotation Time:       ',crt)
print('Observers Longitude [deg]:      ',np.rad2deg(obsLon))
print('Observers Latitude [deg]:       ',np.rad2deg(obsLat))
print('Observers distance to Sun [AU]: ',Obs_Sun_AU)
print('Apparent solar radius [arcsec]: ',rsunarc)

Carrington Rotation Time:        2017-08-19 04:44:52.946
Observers Longitude [deg]:       323.9999982980418
Observers Latitude [deg]:        6.830680503300187
Observers distance to Sun [AU]:  1.012050123894962
Apparent solar radius [arcsec]:  947.8098777838427


In [8]:
## rObs is the radial position of the observer in solar radii
rObs     = Obs_Sun_AU * (1.495978707e11/6.96340e8)   
## Observers latitude in the PSI model geometry 
thetaObs = np.pi/2. - obsLat
## Observers Lon in PSI model geometry
phiObs   = obsLon
## cartesian position of Observer in PSI model geometery (units of solar radii)
xObs,yObs,zObs = rObs*np.sin(thetaObs)*np.cos(phiObs),rObs*np.sin(thetaObs)*np.sin(phiObs),rObs*np.cos(thetaObs)
print(xObs,yObs,zObs)

174.65060561060434 -126.89110044448852 25.859384238598253


### Synthesized Field-of-View Size and Sampling

- The synthesis geometry will have the x axis pointed toward the observer.  Z is aligned with polar axis. 

In [9]:
yfov_range = (-3.*rsunarc,+3*rsunarc)
zfov_range = (-3.*rsunarc,+3*rsunarc)
arcsamp    = 20.  ## sampling in arcsecond
nysamp = int(np.ceil((yfov_range[1]-yfov_range[0])/arcsamp))
nzsamp = int(np.ceil((zfov_range[1]-zfov_range[0])/arcsamp))
yarc = np.linspace(yfov_range[0],yfov_range[1],nysamp)
zarc = np.linspace(zfov_range[0],zfov_range[1],nzsamp)
yya,zza  = np.meshgrid(yarc,zarc,indexing = 'ij')
rra = np.sqrt(yya**2. + zza**2.) 
mask_ondisk = 1.*(rra>rsunarc)    ## a mask where the field of view cross the solar disk

### Calculate ray samples along each line-of-sight

In [10]:
## First find the cartestian points of each line of sight in the zy plane of the observer's geometry 
xxObs = np.zeros_like(yya)
yyObs = rObs * np.tan(np.deg2rad(yya/3600.))
zzObs = rObs * np.tan(np.deg2rad(zza/3600.))

## rotate these points into the model geometry with Euler rotation
rotm = euler_rz(-phiObs) @ euler_ry(-(thetaObs-np.pi/2.))
xyz_model = rotm.T  @ np.stack((xxObs.flatten(),yyObs.flatten(),zzObs.flatten()))

## NOW WITH THE XYZ_MODEL POINTS AND THE LOCATION OF THE OBSERVER,
## come up with the parametric equations for the los of sight
## and then get the spherical coordinates, interpolate for rho and temps, etc.
## synthesize and integrate for integrated I,Q,U, (V?)

losvec    = np.stack((xObs-xyz_model[0,:],yObs-xyz_model[1,:],zObs - xyz_model[2,:]))
losveclen = np.linalg.norm(losvec,axis=0,ord=2,keepdims = True)
losvec    = losvec / losveclen
startpt   = xyz_model

In [11]:
lons,lats,rs = corona.lons,corona.lats,corona.rs
## minimum angular spacing
m1 = np.min(np.abs(lons - np.roll(lons,1)))
m2 = np.min(np.abs(lons - np.roll(lons,-1)))
m3 = np.min(np.abs(lats - np.roll(lats,1)))
m4 = np.min(np.abs(lats - np.roll(lats,-1)))
dth = np.min([m1,m2,m3,m4])/2.

In [12]:
tharr = np.linspace(-np.pi/2,np.pi/2,int(np.ceil(np.pi/dth)))
tan_tharr = np.tan(tharr)
rpt = np.linalg.norm(startpt,axis=0,ord=2)
rpt.shape,startpt.shape,losvec.shape

((81225,), (3, 81225), (3, 81225))

In [64]:
import time 
t0 = time.time()
rays_xyz = []
rays_sph = []
dls = []
pts = 0 
for pt in range(len(rpt)):
    
    ## figure out step size using angular coordinates
    dl = tan_tharr*rpt[pt]  ## the length between points 
    
    rayxyz = startpt[:,pt] + losvec[:,pt]*dl[:,None]
    rayr = np.linalg.norm(rayxyz,axis=1)
    
    ## remove those outside of the simulation domain 
    ## to do this, I set the values of the ray to infinity so that the bound 
    ## check of the interpolator below will return 0. 
    ## Doing it this way always the rays to be efficiencyly stacked 

    rm = np.linalg.norm(rayxyz,axis=1)
    tm = np.arccos((rayxyz[:,2]/rm))
    pm = np.arctan2(rayxyz[:,1],rayxyz[:,0])
    pm[pm<0] += 2*np.pi  ## so that it falls in the simulation domain 

    ray_sp = np.stack([rm,tm,pm]).T  ## the spherical coordinates for each point along the ray 
       
    rayxyz[rayr>rs.max(),:] = np.inf
    ray_sp[rayr>rs.max(),:] = np.inf
    dl[rayr>rs.max()]  = 0.    
        
    pts += len(ray_sp[0,:])
    rays_sph.append(ray_sp.T)
    rays_xyz.append(rayxyz.T)
    dls.append(dl)
    
t1 = time.time()
print(t1-t0)

/tmp/ipykernel_179783/2395897343.py:21: RuntimeWarning: invalid value encountered in true_divide
  tm = np.arccos((rayxyz[:,2]/rm))


8.851547718048096


In [65]:
rayxyz.shape,ray_sp.shape

((286, 3), (286, 3))

In [66]:
rays_sph = np.array(rays_sph).reshape(len(rpt)*len(tan_tharr),3)
rays_xyz = np.array(rays_xyz).reshape(len(rpt)*len(tan_tharr),3)

In [67]:
rays_sph.shape,rays_xyz.shape

((23230350, 3), (23230350, 3))

In [68]:
#### Get interpolating functions for all necessary simulation data
ccoefInt  = rgi((lons,lats,rs),ccoef_10747,method = 'linear',fill_value = 0.,bounds_error = False)
alignInt  = rgi((lons,lats,rs),align_10747,method = 'linear',fill_value = 0.,bounds_error = False)
bxInt   = rgi((lons,lats,rs),corona.bx,method = 'linear',fill_value = 0.,bounds_error = False)
byInt   = rgi((lons,lats,rs),corona.by,method = 'linear',fill_value = 0.,bounds_error = False)
bzInt   = rgi((lons,lats,rs),corona.bz,method = 'linear',fill_value = 0.,bounds_error = False)

## add velocities! 

In [69]:
%time bxrays = bxInt(rays_sph) 

CPU times: user 5.72 s, sys: 1.7 s, total: 7.42 s
Wall time: 7.42 s


In [70]:
%time byrays = byInt(rays_sph) 

CPU times: user 5.74 s, sys: 1.69 s, total: 7.43 s
Wall time: 7.43 s


In [71]:
%time bzrays = bzInt(rays_sph) 

CPU times: user 5.74 s, sys: 1.7 s, total: 7.44 s
Wall time: 7.43 s


In [72]:
## get ThetaB (inclination of B wrt to LOS)
bxyzs = np.stack((bxrays,byrays,bzrays)).T
blens = np.linalg.norm(bxyzs,axis =1)
bxyzs = bxyzs.reshape(len(rpt),len(tan_tharr),3)
blens = blens.reshape(len(rpt),len(tan_tharr))
bxyzs = np.swapaxes(np.swapaxes(bxyzs,0,2),0,1)

In [73]:
bxyzs.shape,losvec.shape,blens.T.shape

((286, 3, 81225), (3, 81225), (286, 81225))

In [74]:
thetaBlos = np.arccos( (np.sum(bxyzs*losvec[None,:,:],axis=1)/blens.T).clip(min = -1,max=1))
thetaBlos[np.isnan(thetaBlos)] = 0.

/tmp/ipykernel_179783/4076082180.py:1: RuntimeWarning: invalid value encountered in true_divide
  thetaBlos = np.arccos( (np.sum(bxyzs*losvec[None,:,:],axis=1)/blens.T).clip(min = -1,max=1))


In [77]:
## angle between losvec and the vector from disk center
rlens = np.linalg.norm(rays_xyz,axis=1)
rlens = rlens.reshape(len(rpt),len(tan_tharr)).T
rays_xyz = rays_xyz.reshape(len(rpt),len(tan_tharr),3)
rays_xyz = np.swapaxes(np.swapaxes(rays_xyz,0,2),0,1)
thetaDClos = np.arccos((np.sum(rays_xyz*losvec[None,:,:],axis=0)/rlens).clip(min = -1,max=1))

ValueError: operands could not be broadcast together with shapes (3,81225) (286,81225) 

In [76]:
rays_xyz.shape,losvec.shape,rlens.shape

((286, 3, 81225), (3, 81225), (81225, 286))

In [ ]:
## angle between losvec and the vector from disk center
rlens = np.linalg.norm(xyz_next,axis=0)
thetaDClos = np.arccos((np.sum(xyz_next*losvec,axis=0)/rlens).clip(min = -1,max=1))


In [95]:



## angle between losvec and the vector from disk center
rlens = np.linalg.norm(xyz_next,axis=0)
thetaDClos = np.arccos((np.sum(xyz_next*losvec,axis=0)/rlens).clip(min = -1,max=1))

## get projection of B onto plane perpendicular to LOS
## and the projection of DC vector onto same plane
Bperp  = bxyzs -  (blens*np.cos(thetaBlos))*losvec
DCperp = xyz_next - (rlens*np.cos(thetaDClos))*losvec

## find angle between Bperp and DCperp, which is the azimuthal angle relative to disk center
costhetaAzi = np.sum((Bperp*DCperp),axis=0)/(np.linalg.norm(Bperp,axis = 0)*np.linalg.norm(DCperp,axis = 0))
thetaAzi = np.arccos(costhetaAzi.clip(min =-1,max=1))
thetaAzi[np.isnan(thetaAzi)] = 0.

In [ ]:
ln1 = fe13.get_emissionLine(10747.)
geff,D_coeff,E_coeff = ln1.geff,ln1.Dcoeff,ln1.Ecoeff

In [117]:

## SYNTHESIS
totIQU = np.zeros((xxObs.shape[0],xxObs.shape[1],4))
steps = np.linspace(-3,3,np.int(np.ceil(6./(10./960.))))  ## radii units
for nstep,losstep in enumerate(steps):
    print(nstep,len(steps))

    ## get points along line of sight in cartesian and spherical coords
    xyz_next = startpt + losvec*losstep
    rm = np.sqrt(np.sum(xyz_next**2,axis = 0))
    tm = np.arccos(xyz_next[2,:]/rm)
    pm = np.arctan2(xyz_next[1,:],xyz_next[0,:])
    pm[pm<0] += 2*np.pi  ## so that it falls in the simulation domain 
    flat = np.array([pm,tm,rm])
    
    ## get ThetaB (inclination of B wrt to LOS)
    bxyzs = np.stack((bxInt(flat.T),byInt(flat.T),bzInt(flat.T)))
    blens = np.linalg.norm(bxyzs,axis =0)

    ## losvec len is 1
    thetaBlos = np.arccos((np.sum(bxyzs*losvec,axis=0)/blens).clip(min = -1,max=1))
    thetaBlos[np.isnan(thetaBlos)] = 0.

    ## angle between losvec and the vector from disk center
    rlens = np.linalg.norm(xyz_next,axis=0)
    thetaDClos = np.arccos((np.sum(xyz_next*losvec,axis=0)/rlens).clip(min = -1,max=1))

    ## get projection of B onto plane perpendicular to LOS
    ## and the projection of DC vector onto same plane
    Bperp  = bxyzs -  (blens*np.cos(thetaBlos))*losvec
    DCperp = xyz_next - (rlens*np.cos(thetaDClos))*losvec

    ## find angle between Bperp and DCperp, which is the azimuthal angle relative to disk center
    costhetaAzi = np.sum((Bperp*DCperp),axis=0)/(np.linalg.norm(Bperp,axis = 0)*np.linalg.norm(DCperp,axis = 0))
    thetaAzi = np.arccos(costhetaAzi.clip(min =-1,max=1))
    thetaAzi[np.isnan(thetaAzi)] = 0.
    
    ## replace this with a call to the emissionLine class?? 

    ## total population and atomic alignment
    ## this is interpolating both totn and rho...not necessary...multiply them first? 
    
    C_coeff = ccoefInt(flat.T)
    sigma   = alignInt(flat.T)

    ## STOKES COEFFICIENTS
    epsI   = C_coeff*(1.0+(1./(2.*np.sqrt(2.)))*(3.*np.cos(thetaBlos)**2 - 1.)*D_coeff*sigma)
    epsQnr = C_coeff*(3./(2.*np.sqrt(2.)))*(np.sin(thetaBlos)**2)*D_coeff*sigma
    epsQ   = np.cos(2.*thetaAzi)*epsQnr
    epsU   = -np.sin(2.*thetaAzi)*epsQnr
    epsV   = C_coeff*np.cos(thetaBlos)*(1399612.2*blens)*(geff + E_coeff*sigma)
    ## is it behind the Sun?

    if (losstep < 0):
        epsI = epsI.reshape(xxObs.shape)*mask_ondisk
        epsQ = epsQ.reshape(xxObs.shape)*mask_ondisk
        epsU = epsU.reshape(xxObs.shape)*mask_ondisk
        epsV = epsV.reshape(xxObs.shape)*mask_ondisk
    else:
        epsI = epsI.reshape(xxObs.shape)
        epsQ = epsQ.reshape(xxObs.shape)
        epsU = epsU.reshape(xxObs.shape)
        epsV = epsV.reshape(xxObs.shape)
    ###

    totIQU[:,:,0] += epsI
    totIQU[:,:,1] += epsQ
    totIQU[:,:,2] += epsU
    totIQU[:,:,3] += epsV

/tmp/ipykernel_174817/3144194238.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  steps = np.linspace(-3,3,np.int(np.ceil(6./(10./960.))))  ## radii units


0 576
1 576
2 576
3 576
4 576
5 576
6 576
7 576
8 576
9 576
10 576
11 576
12 576
13 576
14 576
15 576
16 576
17 576
18 576
19 576
20 576
21 576
22 576
23 576
24 576
25 576
26 576
27 576
28 576
29 576
30 576
31 576
32 576
33 576
34 576
35 576
36 576
37 576
38 576
39 576
40 576
41 576
42 576
43 576
44 576
45 576
46 576
47 576
48 576
49 576
50 576
51 576
52 576
53 576
54 576
55 576
56 576
57 576
58 576
59 576
60 576
61 576
62 576
63 576
64 576
65 576
66 576
67 576
68 576
69 576
70 576
71 576
72 576
73 576
74 576
75 576
76 576
77 576
78 576
79 576
80 576
81 576
82 576
83 576
84 576
85 576
86 576
87 576
88 576
89 576
90 576
91 576
92 576
93 576
94 576
95 576
96 576
97 576
98 576
99 576
100 576
101 576
102 576
103 576
104 576
105 576
106 576
107 576
108 576
109 576
110 576
111 576
112 576
113 576
114 576
115 576
116 576
117 576
118 576
119 576
120 576
121 576
122 576
123 576
124 576
125 576
126 576
127 576
128 576
129 576
130 576
131 576
132 576
133 576
134 576
135 576
136 576
137 576
138 57

/tmp/ipykernel_174817/3144194238.py:21: RuntimeWarning: invalid value encountered in true_divide
  thetaBlos = np.arccos((np.sum(bxyzs*losvec,axis=0)/blens).clip(min = -1,max=1))
/tmp/ipykernel_174817/3144194238.py:34: RuntimeWarning: invalid value encountered in true_divide
  costhetaAzi = np.sum((Bperp*DCperp),axis=0)/(np.linalg.norm(Bperp,axis = 0)*np.linalg.norm(DCperp,axis = 0))


193 576
194 576
195 576
196 576
197 576
198 576
199 576
200 576
201 576
202 576
203 576
204 576
205 576
206 576
207 576
208 576
209 576
210 576
211 576
212 576
213 576
214 576
215 576
216 576
217 576
218 576
219 576
220 576
221 576
222 576
223 576
224 576
225 576
226 576
227 576
228 576
229 576
230 576
231 576
232 576
233 576
234 576
235 576
236 576
237 576
238 576
239 576
240 576
241 576
242 576
243 576
244 576
245 576
246 576
247 576
248 576
249 576
250 576
251 576
252 576
253 576
254 576
255 576
256 576
257 576
258 576
259 576
260 576
261 576
262 576
263 576
264 576
265 576
266 576
267 576
268 576
269 576
270 576
271 576
272 576
273 576
274 576
275 576
276 576
277 576
278 576
279 576
280 576
281 576
282 576
283 576
284 576
285 576
286 576
287 576
288 576
289 576
290 576
291 576
292 576
293 576
294 576
295 576
296 576
297 576
298 576
299 576
300 576
301 576
302 576
303 576
304 576
305 576
306 576
307 576
308 576
309 576
310 576
311 576
312 576
313 576
314 576
315 576
316 576
317 576


In [119]:
plt.figure()
plt.imshow(np.log10(totIQU[:,:,0]).T,cmap = plt.get_cmap('Greys_r'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
## polarized spectra ??

In [ ]:
## K corona? 

In [ ]:
## AIA intensities? 